In [ ]:
%config Completer.use_jedi = False

## Train

In [ ]:
!PYTHONPATH=$(pwd) python ./scripts/retinanet_train.py --lr 0.00001 --max_epoch 100 --batch_size 4 --tag base_retinanet_pretrained --log_dir $(pwd)/experiments --train_percent .7 --use_p_of_data 1 --data_dir $(pwd)/../dataset

In [ ]:
!pwd

In [ ]:
import torchvision
import torch

## Dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
import os
import cv2
from retinanet.datasets.bird import BirdDataset

In [ ]:
ds_notf = BirdDataset(image_dir="../dataset/data", annotations_dir="../dataset/ann")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
item = ds_notf[1]
for box in item[1]["boxes"]:
    cv2.rectangle(item[0], (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
plt.imshow(item[0])
plt.show()

In [ ]:
from retinanet.datasets.transforms import *
from retinanet.datasets.utils import TransformDatasetWrapper

In [ ]:
transform = Compose(
    [
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

In [ ]:
ds = TransformDatasetWrapper(ds_notf, transform)

## Evaluation

In [ ]:
from retinanet.model.detection.retinanet import retinanet_resnet50_fpn

In [ ]:
model = retinanet_resnet50_fpn(num_classes=2, pretrained=False, pretrained_backbone=False)

In [ ]:
model.load_state_dict(torch.load("/workspace8/RetinaNet/experiments/checkpoints/chpt_base_retinanet.pth"))

In [ ]:
model.eval()

predicted = model([ds[2][0]])

In [ ]:
predicted

In [ ]:
keep = torchvision.ops.nms(predicted[0]["boxes"], predicted[0]["scores"], 0.5)

In [ ]:
item = ds_notf[2]
img = item[0]
oboxes = item[1]["boxes"]

keep = keep.numpy()
boxes = list(np.floor(predicted[0]["boxes"].detach().numpy()[keep]))
scores = list(predicted[0]["scores"].detach().numpy()[keep])

print(len(oboxes))
for box, score in zip(boxes, scores):
    if score > 0.2:
        cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 0, 255), 2)
        
plt.imshow(img,cmap='gray')
plt.show()